In [ ]:
from pycaret.utils import version
version()

In [ ]:
import pandas as pd
data = pd.read_csv('./data/cs-training.csv')

In [ ]:
# initializing setup
from pycaret.classification import *
clf2 = setup(data, target='SeriousDlqin2yrs', numeric_features=['NumberOfTime30-59DaysPastDueNotWorse',
                                                                'NumberOfTimes90DaysLate',
                                                                'NumberOfTime60-89DaysPastDueNotWorse',
                                                                'NumberOfDependents'],
             fix_imbalance = True)

# compare all baseline models and select top 5
top5 = compare_models(n_select=5, sort='AUC', )

In [ ]:
# tune top 5 base models
tuned_top5 = [tune_model(i) for i in top5]

In [ ]:
tuned_top = tune_model(top5[0])

In [ ]:
bagged_top5 = [ensemble_model(i) for i in tuned_top5]

In [ ]:
blender = blend_models(estimator_list=top5)

In [ ]:
best = automl(optimize='AUC')

In [ ]:
final = finalize_model(best)
final

In [ ]:
# ensemble top 5 tuned models
bagged_top5 = [ensemble_model(i) for i in tuned_top5]

# blend top 5 base models
blender = blend_models(estimator_list = top5)

# select best model
best = automl(optimize = 'AUC')

# finalize model
final_clf = finalize_model(best)

In [ ]:
# prediction
test_data = pd.read_csv('data/cs-test.csv')
predictions = predict_model(final, data=test_data)
predictions

In [ ]:
import numpy as np
submission_scores = predictions['Score']
submission_scores
ids = np.arange(1, 101504)
submission = pd.DataFrame( {'Id': ids, 'Probability': submission_scores})
submission.to_csv('submission_pycaret.csv', index=False)

In [ ]:
!mlflow ui

In [ ]:
from sklearn_porter import Porter

porter = Porter(final, language='js')
output = porter.export(embed_data=True)

print(output)

# with open('AdaBoostClassifier.js', 'w') as f:
#     f.write(output)